In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

from dataloader import get_Dataframe, applyCut, applyCuts
from unfold import weighted_binary_crossentropy, multifold

import os
os.environ['CUDA_VISIBLE_DEVICES']="1"

import tensorflow as tf
import tensorflow.keras.backend as K
tf.random.set_seed(1234)
np.random.seed(1234)

#physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

/home/miguel/.local/lib/python3.8/site-packages/uproot3/__init__.py:127: FutureWarning: Consider switching from 'uproot3' to 'uproot', since the new interface became the default in 2020.

    pip install -U uproot

In Python:

    >>> import uproot
    >>> with uproot.open(...) as file:
    ...

  warnings.warn(
/home/miguel/.local/lib/python3.8/site-packages/awkward0/__init__.py:12: FutureWarning: Consider switching from 'awkward0' to 'awkward', since the new interface became the default in 2020.

    pip install -U awkward

In Python:

    >>> import awkward as ak
    >>> new_style_array = ak.from_awkward0(old_style_array)
    >>> old_style_array = ak.to_awkward0(new_style_array)

  warnings.warn(


In [7]:
import gc

In [2]:
#path = '/data/h1/22-08-17-20-57-August17_v2/out_ep0607/'
#path = '/data/h1/22-08-21-08-26-August20_22/out_ep0607/'
path = '/data/h1/22-08-22-06-53-August21/out_ep0607/'

In [3]:
mc_names = ['Rapgap']#,'Rapgap','Rapgap','Rapgap','Rapgap','Rapgap'] 
mc_tags = ['nominal']#,'sys_0','sys_1','sys_5','sys_7','sys_11']
mc_tags_out = ['nominal']#,'sys_0','sys_1','sys_5','sys_7','sys_11']

mc_names += ['Django']#,'Django','Django','Django','Django','Django'] 
mc_tags += ['nominal']#,'sys_0','sys_1','sys_5','sys_7','sys_11']
mc_tags_out += ['nominal']#,'sys_0','sys_1','sys_5','sys_7','sys_11']

In [4]:
reload = True#need to set this to true once, then much faster the second time.
rerun = True

In [5]:
import uproot as ur
temp_file = ur.open(path+'Rapgap_Eplus0607_10.nominal.root')
print(temp_file)
#temp_file.keys()
#temp_file['Data']['minitree'].keys()
temp_file = temp_file['Rapgap']['minitree']
temp_file.keys()

<ReadOnlyDirectory '/' at 0x7fb9995deca0>


['wgt',
 'event_y_e',
 'event_Q2_e',
 'event_x_e',
 'event_y_s',
 'event_Q2_s',
 'event_x_s',
 'event_y_es',
 'event_Q2_es',
 'event_x_es',
 'event_y_da',
 'event_Q2_da',
 'event_y_h',
 'event_Q2_h',
 'gen_x',
 'gen_y',
 'gen_Q2',
 'gen_event_Q2_es',
 'gen_event_y_es',
 'gen_event_x_es',
 'gen_event_Q2_s',
 'gen_event_y_s',
 'gen_event_x_s',
 'gen_event_Q2_e',
 'gen_event_y_e',
 'gen_event_x_e',
 'vertex_z',
 'ptmiss',
 'pth',
 'ptratio_da',
 'ptratio_ele',
 'acoplanarity',
 'Empz',
 'e_px',
 'e_py',
 'e_pz',
 'e_eta',
 'HFS_px',
 'HFS_py',
 'HFS_pz',
 'HFS_E',
 'HFS_eta',
 'genHFS_px',
 'genHFS_py',
 'genHFS_pz',
 'genHFS_E',
 'genHFS_eta',
 'gene_px',
 'gene_py',
 'gene_pz',
 'gene_eta']

In [8]:
if (reload):
    data = get_Dataframe(path, name='Data', tag='nominal', verbose=True)
    print(data.keys())
    data = applyCuts(data,isMC=False,verbose=True)
    print(len(data))
    data.to_pickle("datafiles/data.pkl")
else:
    data = pd.read_pickle("datafiles/data.pkl")
    
gc.collect()

filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Data_Eplus0607_3.nominal.root
[b'Data;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Data_Eplus0607_1.nominal.root
[b'Data;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Data_Eplus0607_2.nominal.root
[b'Data;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Data_Eplus0607_0.nominal.root
[b'Data;1']
####################################################################
Dataframe has a total of  355805  entries
####################################################################
Index(['event_y_e', 'event_Q2_e', 'event_x_e', 'event_y_s', 'event_Q2_s',
       'event_x_s', 'event_y_es', 'event_Q2_es', 'event_x_es', 'event_y_da',
       'event_Q2_da', 'event_y_h', 'event_Q2_h', 'e_px', 'e_py', 'e_pz',
       'e_eta', 'HFS_px', 'HFS_py', 'HFS_pz', 'HFS_E', 'HFS_eta', 'wgt',
       'vertex_z', 'ptmiss', 'Empz'],
      dtype='object')
ptmiss<10 355805  fraction kept: 97.2
Empz<65 345888  fraction kept: 10

5848

In [9]:
if (reload):
    for mc_i in range(len(mc_names)):
    #Load files
        mc_name = mc_names[mc_i]
        mc_tag = mc_tags[mc_i]
        mc_tag_out = mc_tags_out[mc_i]

        print("on sample:",mc_name,mc_tag)
        mc = get_Dataframe(path, name=mc_name, tag=mc_tag,verbose=True)
        print(mc.keys())

        mc   = applyCuts(mc, isMC=True,verbose=True)
        mc.to_pickle('datafiles/'+mc_name+"_"+mc_tag_out+".pkl")
        del mc
        gc.collect()

on sample: Rapgap nominal
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_119.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_137.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_149.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_104.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_128.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_130.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_112.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_9.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Rapgap_Eplus0607_123.nominal.root
[b'Rapgap;1']
filename is /data/h1/22-08-22-06-53-Augu

filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_127.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_132.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_154.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_114.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_119.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_139.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_123.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_113.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_Eplus0607_109.nominal.root
[b'Django;1']
filename is /data/h1/22-08-22-06-53-August21/out_ep0607/Django_E

In [10]:
import pandas as pd
import numpy as np

In [11]:
for mc_i in range(len(mc_names)):

    #Load files
    mc_name = mc_names[mc_i]
    mc_tag = mc_tags[mc_i]
    mc_tag_out = mc_tags_out[mc_i]
    print (mc_tag_out)
    mc = pd.read_pickle("datafiles/"+mc_name+"_"+mc_tag_out+".pkl")
    pass_truth = np.array(mc['pass_truth'])
    print("on sample:",mc_name,mc_tag,len(mc),sum(pass_truth))

nominal
on sample: Rapgap nominal 61416831 61416831
nominal
on sample: Django nominal 71312483 71312483


In [12]:
data = pd.read_pickle("datafiles/data.pkl")
pass_reco = np.array(data['pass_reco'])
print("on data", len(data),sum(pass_reco))

on data 345888 345888
